## Casus opdracht orientatie

### Lever één Jupyter Notebook in met de volgende onderdelen:

- een eerste verkenning en analyse van je dataset (stappen 1 tot en met 4 uit het data science proces);
- drie concept-onderzoeksvragen;
- een externe dataset (als bijlage in csv-formaat).

### Dataset

- filmgegevens, waaronder duur, genres, taal, land van herkomst, budget en opbrengst;
- likes op facebook voor regisseur, hoofdrolspelers, totale cast en de film zelf;
- score op IMDB en aantal reviews.


### Verplicht onderzoeksvraag
*In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op Facebook en IMDB zelf?*

## Eerste verkenning en analyse van dataset

### Stap 1: Data collection

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_movies = pd.read_csv('movie.csv')
df_movies

### Stap 2: Data processing (Data munging)

In [ ]:
print(df_movies.columns)
df_movies.describe()

In [ ]:
df_movies.dtypes

In [ ]:
# Sort columns 

df_movies = df_movies[['movie_imdb_link', 'movie_title', 'imdb_score', 'title_year', 'director_name', 'director_facebook_likes', 'actor_1_name',
                      'actor_1_facebook_likes', 'actor_2_name', 'actor_2_facebook_likes', 'actor_3_name', 'actor_3_facebook_likes',
                      'cast_total_facebook_likes', 'movie_facebook_likes', 'genres', 'budget', 'gross', 'country', 'language',
                      'num_critic_for_reviews', 'num_user_for_reviews', 'num_voted_users', 'plot_keywords', 'color', 'content_rating',
                      'duration', 'aspect_ratio', 'facenumber_in_poster']]
df_movies

### Stap 3: Data Cleaning

Drop overall duplicates first.

In [ ]:
print('Before removing duplicates', len(df_movies))
df_movies = df_movies.drop_duplicates()
print('After removing duplicates:',len(df_movies))

#### 3.x Kolom: movie_imdb_link

The movie_imdb_link duplicates only differ on a few columns like likes and votes. Extract the unique identifier from the URL and remove these duplicate rows.

In [ ]:
pd.concat(gby_result for _, gby_result in df_movies.groupby("movie_imdb_link") if len(gby_result) > 1)

In [ ]:
df_movies['movie_imdb_link'] = df_movies['movie_imdb_link'].str.extract(r'(?<=title\/)(.*)(?=\/\?)', expand=False)
print('Length before removing duplicates', len(df_movies))
df_movies = df_movies.drop_duplicates(subset='movie_imdb_link')
print('Length after removing duplicates:',len(df_movies))

#### 3.x Kolom: movie_title

Strip whitespaces from both ends for the title. Duplicate movie_title rows might be a remake or a reboot of the movie. Leave them.

In [ ]:
df_movies['movie_title'] = df_movies['movie_title'].str.strip()

#### 3.x Kolom: title_year
Rows that have NaN for title_year are series/reviews, not movies. We won't need these for our analysis. CHange title_year to DateTime64 for time series analysis.

In [ ]:
df_movies.loc[df_movies['title_year'].isnull()]

In [ ]:
print('Length before removing NaN for title_year:', len(df_movies))
df_movies = df_movies.drop(df_movies.loc[df_movies['title_year'].isnull()].index)
print('Length after removing NaN for title_year:',len(df_movies))
df_movies['title_year'] = pd.to_datetime(df_movies['title_year'], format='%Y', errors='coerce')
df_movies

#### 3.x Kolom: actor_1_name
Rows that have NaN for actor_1_name are documentaries, not movies. Remove them.

In [ ]:
df_movies.loc[df_movies['actor_1_name'].isnull()]


In [ ]:
print('Length before removing NaN for actor_1_name:', len(df_movies))
df_movies = df_movies.drop(df_movies.loc[df_movies['actor_1_name'].isnull()].index)
print('Length after removing NaN for actor_1_name:',len(df_movies))
df_movies

#### 3.x Kolom: genres

Genres are split with an '|' delimeter. In total there are 28 unique genres. There are no NaN values. Split them and give them an own boolean column.

In [ ]:
import itertools

list_genres = list(set(itertools.chain.from_iterable(df_movies.genres.str.split('|'))))
print(list_genres)

def add_genre(df, genre):
    genreConcat = 'genre_' + genre
    df_copy = df.copy()
    df_copy[genreConcat] = df_copy['genres'].str.contains(pat = genre)
    return df_copy

for genre in list_genres:
    df_movies = add_genre(df_movies, genre)

df_movies

#### 3.x Kolom plot_keywords
Remove '|' delimeter to able to use text mining (if needed).

In [ ]:
df_movies['plot_keywords'] = df_movies['plot_keywords'].str.replace('|', ' ')
df_movies['plot_keywords']

#### 3.x Kolom content_rating
Replace NaN and 'Unrated' with 'Not Rated'.

In [ ]:
print(df_movies['content_rating'].unique())

df_movies['content_rating'] = df_movies['content_rating'].str.replace('Unrated', 'Not Rated')
df_movies['content_rating'] = df_movies['content_rating'].fillna(value='Not Rated')

print(df_movies['content_rating'].unique())

#### 3.x Kolom color
All rows with NaN on color are released after 1990. Assume color is used (available since 1950s).

In [ ]:
df_movies['color'] = df_movies['color'].fillna(value='Color')
df_movies['color'].unique()

#### 3.x Remove unimportant NaN's

Remove rows that have columns with NaN values. These NaN values can't be filled in by a 'default' value. Leave budget and gross (might turn out to be too much data loss).

In [ ]:
print('Length before removing NaNs', len(df_movies))

cols_to_ignore = ['movie_imdb_link', 'budget', 'gross']
df_budget_gross = df_movies[cols_to_ignore]
df_movies = df_movies.drop(['budget', 'gross'], axis=1)

df_movies = df_movies.dropna()

print('Length after removing NaNs', len(df_movies))

df_movies = df_movies.join(df_budget_gross.set_index('movie_imdb_link'), on='movie_imdb_link')

#### 3.x Change to int64

In [ ]:
df_movies = df_movies.astype({'director_facebook_likes': 'int64',
                            'actor_1_facebook_likes': 'int64',
                            'actor_2_facebook_likes': 'int64',
                            'actor_3_facebook_likes': 'int64',
                            'cast_total_facebook_likes': 'int64',
                            'num_critic_for_reviews': 'int64',
                            'num_user_for_reviews': 'int64',
                            'num_voted_users': 'int64',
                            'duration': 'int64',
                            'facenumber_in_poster': 'int64'})

df_movies

### Stap 4: Data Visualization

In [ ]:
import matplotlib.pyplot as plt

Check and see if there is a correlation between budget and duration. Set a limit on budget to see a clear scatterplot.

In [ ]:
fig = plt.figure(1, figsize=(10,10))

y_budget = df_movies[['budget']]
x_duration = df_movies[['duration']]

axScatter = plt.subplot(111)
axScatter.scatter(x_duration, y_budget)
plt.ylim(0, 300000000)
axScatter.set_title('Scatterplot between budget and duration')
axScatter.set_xlabel('Duration in minutes')
axScatter.set_ylabel('Budget in US Dollars')